# 1. ConversationTokenBufferMemory的使用

## 举例1:

In [10]:
import os
from langchain.memory import ConversationTokenBufferMemory, ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
import dotenv

dotenv.load_dotenv()

chat_model = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("ARK_API_KEY"),
)
# NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model doubao-1-5-pro-32k-250115. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

# 使用字符计数替代 token 计数
memory = ConversationTokenBufferMemory(
    llm=chat_model,
    max_token_limit=10,
)

memory.save_context({"input":"你好吗?"},{"output":"我很好,谢谢!"})
memory.save_context({"input":"今天天气如何?"},{"output":"晴天,25度"})

print(memory.load_memory_variables({}))

NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model doubao-1-5-pro-32k-250115. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

# 2. ConversationSummaryMemory

In [12]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
import dotenv
import os

dotenv.load_dotenv()

chat_model = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("ARK_API_KEY"),
)

memory = ConversationSummaryMemory(llm = chat_model)

memory.save_context({"input":"你好"},{"output":"怎么了"})
memory.save_context({"input":"你是谁?"},{"output":"我是AI助手小智"})
memory.save_context({"input":"初次对话,你能介绍一下你自己吗？"},{"output":"当然可以了,我是一个无所不能的小智"})

print(memory.load_memory_variables({}))

{'history': '人类向AI打招呼说“你好”，AI回应询问“怎么了”。随后人类询问AI“你是谁”，AI回答自己是AI助手小智。之后人类因初次对话让AI介绍一下自己，AI表示当然可以，并称自己是无所不能的小智。 '}


## 举例2: 如果实例化ConversationSummaryMemory前,已经有历史消息,可以调用from_messages()实例化

In [16]:
from langchain_openai import ChatOpenAI
import os
import dotenv
from langchain.memory import ConversationSummaryMemory,ChatMessageHistory

dotenv.load_dotenv()

chat_model = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("ARK_API_KEY"),
)

history = ChatMessageHistory()
history.add_user_message("你好，你是谁？")
history.add_ai_message("我是AI助手小智")

memory = ConversationSummaryMemory.from_messages(
    llm = chat_model,
    #是生成摘要的原材料 保留完整对话供必要时回溯。当新增对话时，LLM需要结合原始历史生成新摘要
    chat_memory = history,
)

print(memory.load_memory_variables({}))

memory.save_context(inputs={"human":"我的名字叫小明"},outputs={"ai":"很高兴认识你"})

print(memory.load_memory_variables({}))

# 记录了历史的交互的消息
print(memory.chat_memory.messages)

{'history': '人类询问“你好，你是谁？”，AI回答“我是AI助手小智”。'}
{'history': '人类询问“你好，你是谁？”，AI回答“我是AI助手小智”。随后人类告知AI自己的名字叫小明，AI表示很高兴认识人类。 '}
[HumanMessage(content='你好，你是谁？', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的名字叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={})]


# 3. ConversationSummaryBufferMemory

## 举例1:

In [18]:
from langchain_openai import ChatOpenAI
import dotenv
import os
from langchain.memory import ConversationSummaryBufferMemory


dotenv.load_dotenv()

llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("ARK_API_KEY"),
)

memory =  ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=40, # 控制缓冲区的大小
    return_messages=True
)

# NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model doubao-1-5-pro-32k-250115. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

# 向memory中存储信息
memory.save_context(inputs={"input":"你好,我的名字叫小明"},outputs={"output":"很高兴认识你"})
memory.save_context({"input": "李白是哪个朝代的诗人"}, {"output": "李白是唐朝诗人"})
memory.save_context({"input": "唐宋八大家里有苏轼吗？"}, {"output": "有"})

print(memory.load_memory_variables({}))

print("\n")

print(memory.chat_memory.messages)






NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model doubao-1-5-pro-32k-250115. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

## 举例2: 模拟客服交互

In [20]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.llm import LLMChain
import dotenv
import os

# 1. 初始化大语言模型（修正模型名称）
dotenv.load_dotenv()

llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("ARK_API_KEY"),
)

# 2. 定义提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是电商客服助手，用中文友好回复用户问题。保持专业但亲切的语气。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# 3. 创建带摘要缓冲的记忆系统
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=400,
    memory_key="chat_history",
    return_messages=True
)

# 4. 创建对话链
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
)

# 5. 模拟多轮对话
dialogue = [
    ("你好，我想查询订单12345的状态", None),
    ("这个订单是上周五下的", None),
    ("我现在急着用，能加急处理吗", None),
    ("等等，我可能记错订单号了，应该是12346", None),
    ("对了，你们退货政策是怎样的", None)
]

# 6. 执行对话
print("=== 对话开始 ===")
for user_input, _ in dialogue:
    response = chain.invoke({"input": user_input})
    print(f"用户: {user_input}")
    print(f"客服: {response['text']}\n")

# 7. 查看当前记忆状态
print("\n=== 当前记忆内容 ===")
memory_content = memory.load_memory_variables({})
print(memory_content['chat_history'])

=== 对话开始 ===


NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model doubao-1-5-pro-32k-250115. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

# 4. ConversationEntityMemory(了解)

In [21]:
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain_openai import ChatOpenAI
import os
import dotenv

# 1. 初始化大语言模型（修正模型名称）
dotenv.load_dotenv()

llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("ARK_API_KEY"),
)
# 2. 初始化实体记忆
memory = ConversationEntityMemory(llm=llm)

# 3. 创建对话链
chain = LLMChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=memory,  # 使用上面创建的memory对象
    verbose=True,  # 开启详细日志
)

# 4. 进行多轮对话
print("=== 对话开始 ===")
response1 = chain.invoke(input="你好，我叫蜘蛛侠。我的好朋友包括钢铁侠、美国队长和绿巨人。")
print(f"AI回复: {response1['text']}")

response2 = chain.invoke(input="我住在纽约。")
print(f"AI回复: {response2['text']}")

response3 = chain.invoke(input="我使用的装备是由斯塔克工业提供的。")
print(f"AI回复: {response3['text']}")

# 5. 查询存储的实体信息
print("\n=== 当前存储的实体信息 ===")
print(memory.entity_store.store)

# 6. 基于记忆提问
print("\n=== 基于记忆的提问 ===")
final_answer = chain.invoke(input="你能告诉我蜘蛛侠住在哪里以及他的好朋友有哪些吗？")
print(f"AI回答: {final_answer['text']}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_39092\2072033208.py:17: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationEntityMemory(llm=llm)
C:\Users\Admin\.conda\envs\py310\lib\site-packages\pydantic\main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


=== 对话开始 ===


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you recei

# 5. ConversationKGMemory(了解)

```bash
pip install networkx
```

In [24]:
# 1. 导入相关包
from langchain.memory import ConversationKGMemory
from langchain_openai import ChatOpenAI
import os
import dotenv

# 1. 初始化大语言模型（修正模型名称）
dotenv.load_dotenv()

llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("ARK_API_KEY"),
)
# 3. 定义ConversationKGMemory对象
memory = ConversationKGMemory(llm=llm)

# 4. 保存会话
memory.save_context({"input": "向山姆问好"}, {"output": "山姆是谁"})
memory.save_context({"input": "山姆是我的朋友"}, {"output": "好的"})

# 5. 查询会话
memory_data = memory.load_memory_variables({"input": "山姆是谁"})
print("=== 知识图谱记忆内容 ===")
print(memory_data)

=== 知识图谱记忆内容 ===
{'history': 'On 山姆: 山姆 是 我的朋友.'}


In [25]:
memory.get_knowledge_triplets("她最喜欢的颜色是红色")

[KnowledgeTriple(subject='她', predicate='最喜欢的颜色是', object_='红色')]

# 6. VectorStoreRetrieverMemory(了解)

In [39]:
import os
import dotenv
from langchain_openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory, ConversationBufferMemory
from langchain_community.vectorstores import FAISS

# 加载环境变量
dotenv.load_dotenv()


# 1. 定义ConversationBufferMemory对象并保存上下文
memory = ConversationBufferMemory()
memory.save_context({"input": "我最喜欢的食物是披萨"}, {"output": "很高兴知道"})
memory.save_context({"input": "我喜欢的运动是跑步"}, {"output": "好的,我知道了"})
memory.save_context({"input": "我最喜欢的运动是足球"}, {"output": "好的,我知道了"})

# 2. 定义向量嵌入模型
# BadRequestError: Error code: 400 - {'error': {'code': 'InvalidParameter', 'message': 'The parameter `input[0]` specified in the request are not valid: expected a string, but got `[35075 25 50534 239 32335 83601 250 25340 95 9554 72406 253 53953 21043 24326 104 81180 101]` instead. Request id: 02176026306822061ee2447fb8673108fd2fdb6d547618f607175', 'param': 'input[0]', 'type': 'BadRequest'}}
embeddings_model = OpenAIEmbeddings(model=os.getenv("MODEL_EMBEDDING"),
                                    api_key=os.getenv("ARK_API_KEY"),
                                    base_url=os.getenv("BASE_URL"),)

# 3. 初始化向量数据库
# 提取对话历史并转换为文本列表
conversation_history = memory.buffer.split("\n")
vectorstore = FAISS.from_texts(conversation_history, embeddings_model)

# 4. 定义检索对象
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))

# 5. 初始化VectorStoreRetrieverMemory
vector_memory = VectorStoreRetrieverMemory(retriever=retriever)

# 6. 查询记忆
result = vector_memory.load_memory_variables({"prompt": "我最喜欢的食物是"})
print("\n=== 检索结果 ===")
print(result)

BadRequestError: Error code: 400 - {'error': {'code': 'InvalidParameter', 'message': 'The parameter `input[0]` specified in the request are not valid: expected a string, but got `[35075 25 50534 239 32335 83601 250 25340 95 9554 72406 253 53953 21043 24326 104 81180 101]` instead. Request id: 02176026306822061ee2447fb8673108fd2fdb6d547618f607175', 'param': 'input[0]', 'type': 'BadRequest'}}